In [1]:
# Useful starting lines
import numpy as np
from helpers import *
from methods import *
from process_data import *
from crossvalidation import *
from select_parameter import *

%load_ext autoreload
%autoreload 2

seed=10

# Load the dataset

In [3]:
from zipfile import ZipFile 
  
# # specifying the zip file name 
file_name = 'Data/test.csv.zip'
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    zip.extractall('Data/') 

In [4]:
y, tX, ids = load_csv_data('Data/train.csv')
_, tX_test, ids_test = load_csv_data('Data/test.csv')

In [29]:
ypred = np.ones(tX.shape[0])

print(list(y[tX[:,4]<46]).count(1))
print(list(y[tX[:,4]<46]).count(-1))
ypred[tX[:,4]>3.2]=-1
print(ypred.shape,y.shape)

compute_accuracy(ypred, y)

85667
164333
(250000,) (250000,)


0.307776

In [47]:
xtrain= tX[tX[:,22]==0]
ypred = -np.ones(xtrain.shape[0])
ytrain=  y[tX[:,22]==0]
#ypred[xtrain[:,1]<27]=1
compute_accuracy(ypred, ytrain)

0.7448580264830402

In [ ]:
c1=np.count_nonzero(tX[:,22]==0)
c2=np.count_nonzero(tX[:,22]==1)
c3=np.count_nonzero(tX[:,22]==2)
c4=np.count_nonzero(tX[:,22]==3)

print(c1,c2,c3,c4)

99913 77544 50379 22164


# Methods

## 1. Least Squares with Gradient Descent

#### Cross Validation

In [ ]:
# Degree polynomial expansion
degrees = [10,10,10,10]

# Model parameters
max_iters = 5000
gamma = 0.005


# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation(y, tX, least_squares_GD, _, k_indices, k, degrees, alpha,
                                           max_iters=max_iters, gamma=gamma)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0 - Training accuracy: 0.780448 / Test accuracy : 0.781016
1 - Training accuracy: 0.779968 / Test accuracy : 0.778256

Average test accuracy: 0.779636
Variance test accuracy: 0.000002
Min test accuracy: 0.778256
Max test accuracy: 0.781016


## 2. Least Squares with Stochastic Gradient Descent

#### Cross Validation

In [ ]:
# Degree polynomial expansion
degrees = [10,10,10,10]

# Model parameters
max_iters = 1000
gamma = 0.005
batch_size=1


# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation(y, tX, least_squares_SGD, _, k_indices, k, degrees, alpha, 
                                           max_iters=max_iters, gamma=gamma, batch_size=batch_size)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

## 3. Least Squares with Normal Equations 

#### Search for hyperparameters: Degrees.

In [ ]:
# Model parameters for least squares
#tuning parameters for each category
degrees_candidates = [9,10,11,12,13,14]
alpha = 0
k_fold = 3
par_degree, par_lambda, accu = select_parameters_least_squares(y,tX,degrees_candidates,alpha,k_fold,seed)
par_degree, accu

#### Cross Validation

In [228]:
# Degree polynomial expansion
#degrees = [8,11,9]
degrees = [7,10,9,9]
#degrees = [5,7,6,6]
eta=[10,10,14,14]

alpha = 0

# Split data in k-fold
k_fold = 3
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    print(k)
    acc_train, acc_test = cross_validation(y, tX, least_squares, k_indices, k, degrees, alpha)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0


LinAlgError: Singular matrix

## 4. Ridge regression with Normal Equations

#### Search for hyperparameters: Lambdas, Degrees

In [15]:
# Model parameters for ridge regression
#tuning parameters for each category
degrees_candidates = [10]
alphas_candidates=[3]
lambdas_candidates = np.logspace(-3,0,4)
#lambdas_candidates=[1e-5]


k_fold = 2
par_degree, par_lambda, par_alpha, accu = select_parameters_ridge_regression(y,tX,degrees_candidates,lambdas_candidates,
                                                                  alphas_candidates,k_fold,seed)
par_degree, par_lambda, par_alpha, accu

10 0.001 3
0.8551184141600728 0.8502109122990819
10 0.001 3
0.8557800998042513 0.8488599707755508
10 0.01 3
0.8524716715833586 0.8502660527694301
10 0.01 3
0.8544842987510683 0.84919081359764
10 0.1 3
0.8482258553665463 0.8458548151415731
10 0.1 3
0.8501557718287337 0.8462683686691848
10 1.0 3
0.8339169033111853 0.8323729701414353
10 1.0 3
0.8350748531884977 0.83253839155248


([10.0], [0.01], [3.0], [0.8502660527694301])

#### Cross Validation

In [41]:
# Process data parameters
#degrees = [8, 8, 10]
degrees=[9,9,9]
#alphas = [4.0, 5.0, 3.0]
alpha=[3,3,3]

# Model parameters
#lambdas = [1e-06, 0.001, 0.01]
lambdas=[0.00001,0.00001,0.00001]

# Split data in k-fold
k_fold = 3
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    print(k)
    acc_train, acc_test = cross_validation_ridge_regression(y, tX, k_indices, k, lambdas, degrees, alphas)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0
0.8433513734054936 0.840255361021444
1
0.8440233760935044 0.8395953583814335
2
0.843627374509498 0.8406753627014508
0 - Training accuracy: 0.843351 / Test accuracy : 0.840255
1 - Training accuracy: 0.844023 / Test accuracy : 0.839595
2 - Training accuracy: 0.843627 / Test accuracy : 0.840675

Average test accuracy: 0.840175
Variance test accuracy: 0.000000
Min test accuracy: 0.839595
Max test accuracy: 0.840675


## 5. Logistic Regression with Stochastic Gradient Descent


#### Cross Validation

In [ ]:
# Degree polynomial expansion
degrees = [7,10,9,9]

# Model parameters
max_iters = 5000
gamma = 0.005
batch_size = 1

# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation(y, tX, logistic_regression, k_indices, k, batch_size=batch_size, 
                                           max_iters=max_iters, gamma=gamma, degrees=degrees)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

7.997182550126146 0 /5000
0.7524469806959074 1000 /5000
0.4363734465688057 2000 /5000
0.400023839876429 3000 /5000
0.40076385746875537 4000 /5000
10.371044956652037 0 /5000
1.5550707519015254 1000 /5000
0.661663323193932 2000 /5000
0.5936779549816508 3000 /5000
0.5260682969312651 4000 /5000
8.442467521375363 0 /5000
1.7929335626679517 1000 /5000
0.7223707415043099 2000 /5000
0.5914451124506169 3000 /5000
0.5454690010169485 4000 /5000
12.029170671306018 0 /5000
2.0525746152729285 1000 /5000
0.8818547205280977 2000 /5000
0.6671236655053144 3000 /5000
0.5709404738188759 4000 /5000
6.689309726685983 0 /5000
0.7984062586900572 1000 /5000
0.4669079564079578 2000 /5000
0.4200246548981903 3000 /5000
0.4292098025507569 4000 /5000
9.748515728107092 0 /5000
1.2804452477863466 1000 /5000
0.6366004683073873 2000 /5000
0.5702327086476892 3000 /5000
0.5406348712757687 4000 /5000
10.207457083659229 0 /5000
1.8709968020623324 1000 /5000
0.8871537214879525 2000 /5000
0.6549933484898737 3000 /5000
0.6273

## 6. Regularized Logistic Regression with Stochastic Gradient Descent

#### Optimal Lambda

In [ ]:
# TO DO

#### Cross Validation

In [ ]:
# TO DO

lambda_ = 0.001
initial_w = np.random.random(tX.shape[1])
batch_size = 1
max_iters = 1000
gamma = 0.1

loss, weights = reg_logistic_regression(y, tX, lambda_, initial_w, batch_size,  max_iters, gamma)

# Prediction (file.run)
by now the best accuracy predicted is through RIDGE REGRESSION

In [18]:
# Split data in subsets corresponding to a jet value
msks_jet_train = get_jet_masks(tX)
msks_jet_test = get_jet_masks(tX_test)

# Degree polynomial expansion
degrees = [8,8,10]
alphas = [4,5,3]
# Ridge regression parameters for each subset
lambdas = [1e-06, 0.001, 0.01]

# Vector to store the final prediction
y_pred = np.zeros(tX_test.shape[0])

for idx in range(len(msks_jet_train)):
    x_train = tX[msks_jet_train[idx]]
    x_test = tX_test[msks_jet_test[idx]]
    y_train = y[msks_jet_train[idx]]

    # Pre-processing of data
    x_train, x_test = process_data(x_train, x_test, alphas[idx])
    # Trasform the data and add an intercepta
    x_train, x_test = phi(x_train, x_test, degrees[idx])

    loss, weights = ridge_regression(y_train, x_train, lambdas[idx])

    y_test_pred = predict_labels(weights, x_test)

    y_pred[msks_jet_test[idx]] = y_test_pred

In [19]:
higgs = np.count_nonzero(y_pred==1)
print(f'From {y_pred.shape[0]} test examples, {higgs} are 1, i.e. the {higgs/y_pred.shape[0]} %')

From 568238 test examples, 177764 are 1, i.e. the 0.3128337070030515 %


#### Generate predictions and save ouput in csv format for submission

In [20]:
OUTPUT_PATH = 'data/RidgeRegression.csv' 
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [21]:
y_pred1=y_pred

# OTHERS (old)

### Umbalanced Dataset

In [ ]:
higgs = np.count_nonzero(y==1)
print(f'From {y.shape[0]} training examples, {higgs} are 1, i.e. the {higgs/y.shape[0]} %')

# Random Over Sampling
#tX, y = Random_Over_Sampling(tX, y)

#higgs = np.count_nonzero(y==1)
#print(f'Applying Random Over Sampling: \nFrom {y.shape[0]} training examples, {higgs} are 1, i.e. the {higgs/y.shape[0]} %')

# Preprocessing

In [ ]:
tX, tX_test = process_data(tX, tX_test, add_constant_col=True)

# Cross Validation
IDEA: insert CV in each of the methods above

In [ ]:
def cross_validation(y, x, k_indices, k, regression_method, **args):
    """
    Completes k-fold cross-validation using the regression method
    passed as argument.
    """
    # get k'th subgroup in test, others in train
    msk_test = k_indices[k]
    msk_train = np.delete(k_indices, (k), axis=0).ravel()

    x_train = x[msk_train, :]
    x_test = x[msk_test, :]
    y_train = y[msk_train]
    y_test = y[msk_test]

    # data pre-processing
    #x_train, x_test = process_data(x_train, x_test, True)

    # compute weights using given method
    loss, weights = regression_method(y=y_train, tx=x_train, **args)
    
    # predict output for train and test data
    y_train_pred = predict_labels(weights, x_train)
    y_test_pred = predict_labels(weights, x_test)
    
    
    # compute accuracy for train and test data
    acc_train = compute_accuracy(y_train_pred, y_train)
    acc_test = compute_accuracy(y_test_pred, y_test)

    return acc_train, acc_test

In [ ]:
regression_method = ridge_regression

# Model parameters
lambda_ = 0.0005

# Split data in k-fold
k_fold = 2
k_indices = build_k_indices(y, k_fold, seed)


accs_train = []
accs_test = []

for k in range(k_fold):
    acc_train, acc_test = cross_validation(y, tX, k_indices, k, regression_method, lambda_=lambda_)
    accs_train.append(acc_train)
    accs_test.append(acc_test)
    
for i in range(len(accs_train)):
    print("%d - Training accuracy: %f / Test accuracy : %f" % (i, accs_train[i], accs_test[i]))

print("\nAverage test accuracy: %f" % np.mean(accs_test))
print("Variance test accuracy: %f" % np.var(accs_test))
print("Min test accuracy: %f" % np.min(accs_test))
print("Max test accuracy: %f" % np.max(accs_test))

0 - Training accuracy: 0.775480 / Test accuracy : 0.776096
1 - Training accuracy: 0.775888 / Test accuracy : 0.774656

Average test accuracy: 0.775376
Variance test accuracy: 0.000001
Min test accuracy: 0.774656
Max test accuracy: 0.776096


In [ ]:
# TO CHECK

# To evaluate the best lambda that minimizes the test error
loss, weights, best_lambda = cross_validation_ridge_regression(y,tX)

In [ ]:
# Only for non logistic methods
y_pred = predict_labels(weights, tX_test)

In [ ]:
# Only for Logistic methods
y_pred = sigmoid(tX_test@weights)
y_pred[y_pred <0.5] = -1
y_pred[y_pred > 0.5] = 1